In [1]:
from sklearn.utils import shuffle
import pandas as pd
train=pd.read_csv('./all.csv', engine='python')
train=shuffle(train, random_state=1)
train

,ratio,1659.78516,1658.79688,1657.80957,1656.82129,1655.83301,1654.84473,1653.85644,1652.86719,1651.87793,...,549.72363,548.48535,547.24805,546.00977,544.77148,543.53223,542.29395,541.05371,539.81445,538.57422
12633,0.6,0.005098,0.010038,0.014408,0.018209,0.021441,0.024103,0.026196,0.027719,0.028673,...,-0.006858,-0.004017,-0.000258,0.004419,0.010014,0.016527,0.023958,0.032307,0.041574,0.051759
7583,0.3,-0.003269,-0.000298,0.002317,0.004573,0.006473,0.008015,0.009201,0.010028,0.010499,...,-0.004965,-0.000383,0.005372,0.012298,0.020396,0.029666,0.040107,0.051720,0.064505,0.078461
6709,0.3,-0.007515,0.000073,0.006797,0.012659,0.017657,0.021791,0.025063,0.027471,0.029016,...,0.034242,0.038501,0.043177,0.048270,0.053780,0.059706,0.066048,0.072807,0.079983,0.087575
7540,0.3,0.020609,0.019762,0.018763,0.017612,0.016308,0.014852,0.013244,0.011484,0.009571,...,-0.005685,-0.003894,-0.001767,0.000695,0.003492,0.006625,0.010093,0.013897,0.018036,0.022510
15125,0.7,-0.002297,-0.001547,-0.000779,0.000005,0.000805,0.001623,0.002458,0.003310,0.004178,...,0.009586,0.010186,0.011391,0.013202,0.015618,0.018639,0.022266,0.026498,0.031335,0.036778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10955,0.5,-0.010654,-0.007203,-0.004072,-0.001261,0.001230,0.003401,0.005253,0.006784,0.007996,...,-0.002436,0.000795,0.004777,0.009510,0.014993,0.021228,0.028213,0.035949,0.044436,0.053673
17289,0.8,-0.004942,-0.001348,0.001885,0.004757,0.007267,0.009416,0.011205,0.012631,0.013697,...,-0.014110,-0.012908,-0.010414,-0.006628,-0.001549,0.004822,0.012486,0.021442,0.031691,0.043232
5192,0.2,0.000183,0.007161,0.013412,0.018935,0.023732,0.027800,0.031142,0.033756,0.035643,...,-0.001322,0.001667,0.005251,0.009430,0.014205,0.019575,0.025541,0.032102,0.039258,0.047010
12172,0.6,0.000932,0.006392,0.011097,0.015045,0.018237,0.020673,0.022352,0.023275,0.023442,...,-0.006694,-0.001643,0.004818,0.012689,0.021970,0.032661,0.044761,0.058272,0.073193,0.089523


In [2]:
from sklearn.preprocessing import StandardScaler
#取第一列为标签，后面所有列为特征
def prepare_x(train):
    ndarray_train=train.values
    features=ndarray_train[:,1:]
    labels=ndarray_train[:,0]
    scaler=StandardScaler().fit(features)
    norm_features=scaler.transform(features)
    return norm_features, labels

import numpy as np
X, Y=prepare_x(train)
#经此之后，输入文件变成标准的标签-特征模式

nb_features = 1015 
X_train = np.zeros((len(X), nb_features, 1))
X_train[:, :, 0] = X[:, :nb_features]
#经此之后，改变输入特征格式

from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(test_size=0.05, random_state=1)
for train_index, test_index in sss.split(X_train, Y):
    X_t, X_testing = X_train[train_index], X_train[test_index]
    Y_t, Y_testing = Y[train_index], Y[test_index]
#控制随机数种子，保证每次训练的测试集都是一样的
from sklearn.model_selection import KFold
kfold = KFold(10, True, 1)
for train_index, test_index in kfold.split(X_t):
    X_train, X_test = X_t[train_index], X_t[test_index]
    Y_train, Y_test = Y_t[train_index], Y_t[test_index]

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, Sequential, layers, regularizers
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, Dropout, MaxPool1D, BatchNormalization, ReLU
tf.random.set_seed(3)

model = Sequential()
model.add(Flatten(input_shape=(1015, 1)))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('linear'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1015)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               520192    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0

In [4]:
import os
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping
Adam=Adam(lr=0.00001)
model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mae'])
checkpoint_save_path = "./checkpoint/No20240321.ckpt"
if os.path.exists(checkpoint_save_path + '.index'):
    print('- - - - - - - load the model- - - - - - -')
    model.load_weights(checkpoint_save_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path, save_weights_only=True, save_best_only=True)
history = model.fit(X_train, Y_train, epochs=500, validation_data=(X_test, Y_test), batch_size=209, callbacks=[cp_callback])

Train on 18810 samples, validate on 2090 samples
Epoch 1/500
18810/18810 [==============================] - 2s 124us/sample - loss: 0.0636 - mae: 0.1904 - val_loss: 0.0337 - val_mae: 0.1451
Epoch 2/500
18810/18810 [==============================] - 1s 72us/sample - loss: 0.0328 - mae: 0.1375 - val_loss: 0.0254 - val_mae: 0.1241
Epoch 3/500
18810/18810 [==============================] - 1s 71us/sample - loss: 0.0248 - mae: 0.1180 - val_loss: 0.0197 - val_mae: 0.1006
Epoch 4/500
18810/18810 [==============================] - 1s 70us/sample - loss: 0.0221 - mae: 0.1095 - val_loss: 0.0209 - val_mae: 0.1086
Epoch 5/500
18810/18810 [==============================] - 1s 73us/sample - loss: 0.0191 - mae: 0.1013 - val_loss: 0.0185 - val_mae: 0.0978
Epoch 6/500
18810/18810 [==============================] - 1s 72us/sample - loss: 0.0172 - mae: 0.0945 - val_loss: 0.0175 - val_mae: 0.0907
Epoch 7/500
18810/18810 [==============================] - 1s 74us/sample - loss: 0.0166 - mae: 0.0911 - val_l

In [5]:
history.history

{'loss': [0.06363666301800144,
  0.032782297643522425,
  0.024830933122171297,
  0.02211057890413536,
  0.019109037021795908,
  0.017208716614792743,
  0.01660662918455071,
  0.015241123497900036,
  0.013819609768688679,
  0.013053658915062745,
  0.012154139128203194,
  0.011888473574072123,
  0.011084735833315386,
  0.010785024463095599,
  0.009902250616707735,
  0.009698417037725449,
  0.009022797256087263,
  0.008623928675014112,
  0.008825496832529704,
  0.008124316659652525,
  0.007645147361068262,
  0.008398333238437772,
  0.007149504621823629,
  0.006950228115440243,
  0.006921132348684801,
  0.007083695966543423,
  0.006814140278018183,
  0.00669567225397461,
  0.00652570896781981,
  0.006204439425427053,
  0.006273812334984541,
  0.005729718180373311,
  0.0052276330405018395,
  0.005233017494902015,
  0.005385148579565188,
  0.005195663289891349,
  0.004997796197939251,
  0.005193755366942949,
  0.006202034494425687,
  0.005313784874872201,
  0.004981308984052804,
  0.00523793

In [6]:
history_df = pd.DataFrame(history.history)
np.savetxt(r'F:/12046.txt',history_df, delimiter=',')


species_prediction=model.predict(X_testing)
np.savetxt(r'F:/12047.txt',Y_testing, delimiter=',')
np.savetxt(r'F:/12048.txt',species_prediction, delimiter=',')

In [7]:
#预测

test=pd.read_csv('./cellRNA.csv', engine='python')

X, Y=prepare_x(test)
#经此之后，输入文件变成标准的标签-特征模式

nb_features = 1015 
X_test = np.zeros((len(X), nb_features, 1))
X_test[:, :, 0] = X[:, :nb_features]
#经此之后，改变输入特征格式

species_prediction=model.predict(X_test)
np.savetxt(r'F:/0220.txt',species_prediction, delimiter=',')
test

,con,1659.88574,1658.89844,1657.91016,1656.92285,1655.93457,1654.94629,1653.95703,1652.96875,1651.97949,...,549.84863,548.61035,547.37305,546.13477,544.89648,543.65723,542.41895,541.17969,539.93945,538.7002
0,0.0,-0.004729,0.026394,0.054909,0.080820,0.104126,0.124826,0.142922,0.158416,0.171306,...,0.123257,0.125757,0.129259,0.133763,0.139267,0.145774,0.153283,0.161795,0.171308,0.181823
1,0.0,0.035686,0.057156,0.076754,0.094484,0.110344,0.124332,0.136452,0.146702,0.155081,...,0.152689,0.155453,0.159072,0.163545,0.168871,0.175054,0.182090,0.189982,0.198728,0.208331
2,0.0,0.010838,0.025590,0.038850,0.050619,0.060895,0.069678,0.076970,0.082770,0.087077,...,0.071255,0.074881,0.078365,0.081705,0.084901,0.087954,0.090864,0.093631,0.096254,0.098735
3,0.0,-0.000351,0.008617,0.016806,0.024219,0.030853,0.036710,0.041789,0.046091,0.049615,...,0.046010,0.046660,0.047307,0.047953,0.048596,0.049237,0.049876,0.050513,0.051148,0.051782
4,0.0,0.034040,0.036216,0.037904,0.039106,0.039820,0.040046,0.039787,0.039040,0.037806,...,0.099963,0.095002,0.089661,0.083937,0.077833,0.071348,0.064483,0.057238,0.049611,0.041604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#线性回归做对照
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

model = LinearRegression()  

obtain_model=model.fit(X_t,Y_t)

test_predict_label = obtain_model.predict(X_testing)

test_predict_label = np.array(test_predict_label)

np.savetxt(r'F:/12131.txt',test_predict_label, delimiter=',')
np.savetxt(r'F:/12132.txt',Y_testing, delimiter=',')

mae = mean_absolute_error(Y_testing, test_predict_label)
mse = mean_squared_error(Y_testing, test_predict_label)

print(f'线性回归的MAE: {mae}')
print(f'线性回归的MSE: {mse}')

ValueError: Found array with dim 3. Estimator expected <= 2.

In [ ]:
#pls-da回归做对照
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV

plsda = PLSRegression()

n_components_values = list(np.arange(1, 101))

param_grid = {'n_components': n_components_values}

grid_search = GridSearchCV(plsda, param_grid, scoring='neg_mean_squared_error', cv=10)

# 执行网格搜索
grid_search.fit(X_t, Y_t)

# 获取最佳主成分数量
best_n_components = grid_search.best_params_['n_components']

print(f'最佳主成分数量: {best_n_components}')

# 使用最佳参数重新训练模型
best_plsda = PLSRegression(n_components=best_n_components)
best_plsda.fit(X_t, Y_t)

# 在测试集上进行预测
y_pred = best_plsda.predict(X_testing)

np.savetxt(r'F:/12135.txt',y_pred, delimiter=',')
np.savetxt(r'F:/12136.txt',Y_testing, delimiter=',')

mae = mean_absolute_error(Y_testing, y_pred)
mse = mean_squared_error(Y_testing, y_pred)

print(f'线性回归的MAE: {mae}')
print(f'线性回归的MSE: {mse}')